# Semi-Supervised Learning: Manual Label Spreading on Anime Endings

This notebook demonstrates how we predict whether an anime has a **happy** or **sad** ending
using a **manual implementation of label spreading**.

We start with:
- ~12,000 anime samples from public Kaggle Anime Dataset
- Only 50 manually labelled titles
- A similarity graph built from genre, type, rating, members, episodes

We propagate labels through the graph using a custom implementation
(no sklearn allowed).


In [1]:
from pathlib import Path
import sys

p = Path.cwd().resolve()
while p != p.parent and not (p / "src").exists():
    p = p.parent
sys.path.insert(0, str(p))

REPO_ROOT = p
print("Repo root:", REPO_ROOT)



Repo root: /Users/kate/Documents/Machine Learning/anime-label-ML


In [2]:
import pandas as pd
import numpy as np
from src.full_pipeline import run_full_pipeline
from src.config import ANIME_CSV, SEED_LABELS, OUTDIR, K, ALPHA, CHUNK_SIZE, TEST_LABELS

In [3]:
run_full_pipeline(
    anime_csv=str(ANIME_CSV),
    seed_labels_csv=str(SEED_LABELS),
    outdir=str(OUTDIR),
    k=K,
    alpha=ALPHA,
    chunk_size=CHUNK_SIZE,
)


1) Loading & preprocessing features...
Loaded shape: (12294, 7)
Columns: ['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members']
   Feature matrix: (12294, 55)
2) Loading seed labels...
   Seed labels matched: 50 / 50
3) Building chunked kNN-RBF graph W...
Processing rows 0 to 199
Processing rows 200 to 399
Processing rows 400 to 599
Processing rows 600 to 799
Processing rows 800 to 999
Processing rows 1000 to 1199
Processing rows 1200 to 1399
Processing rows 1400 to 1599
Processing rows 1600 to 1799
Processing rows 1800 to 1999
Processing rows 2000 to 2199
Processing rows 2200 to 2399
Processing rows 2400 to 2599
Processing rows 2600 to 2799
Processing rows 2800 to 2999
Processing rows 3000 to 3199
Processing rows 3200 to 3399
Processing rows 3400 to 3599
Processing rows 3600 to 3799
Processing rows 3800 to 3999
Processing rows 4000 to 4199
Processing rows 4200 to 4399
Processing rows 4400 to 4599
Processing rows 4600 to 4799
Processing rows 4800 to 4999
Processing rows

,name,pred,confidence
0,Kimi no Na wa.,1,1.00000
1,Fullmetal Alchemist: Brotherhood,1,1.00000
2,Gintama°,1,1.00000
3,Steins;Gate,1,1.00000
4,Gintama&#039;,0,0.99906
...,...,...,...
12289,Toushindai My Lover: Minami tai Mecha-Minami,1,1.00000
12290,Under World,1,1.00000
12291,Violence Gekiga David no Hoshi,1,1.00000
12292,Violence Gekiga Shin David no Hoshi: Inma Dens...,1,1.00000


In [4]:
preds_path = REPO_ROOT / "outputs" / "predictions.csv"
preds = pd.read_csv(preds_path)
preds.head()

,name,pred,confidence
0,Kimi no Na wa.,1,1.00000
1,Fullmetal Alchemist: Brotherhood,1,1.00000
2,Gintama°,1,1.00000
3,Steins;Gate,1,1.00000
4,Gintama&#039;,0,0.99906


In [5]:
test = pd.read_csv(REPO_ROOT / TEST_LABELS)
m = preds.merge(test[["name","happy_ending"]], on="name", how="inner")

acc = (m["pred"] == m["happy_ending"]).mean()
print("Test accuracy:", round(acc, 3))

cm = np.zeros((2,2), dtype=int)
for t,p in zip(m["happy_ending"], m["pred"]):
    cm[int(t), int(p)] += 1

print("Confusion matrix:\n", cm)
m[["name","happy_ending","pred","confidence"]]


Test accuracy: 0.6
Confusion matrix:
 [[4 1]
 [3 2]]


,name,happy_ending,pred,confidence
0,Kimi no Na wa.,1,1,1.000000
1,Fullmetal Alchemist: Brotherhood,1,1,1.000000
2,Sen to Chihiro no Kamikakushi,1,0,0.855679
3,Cowboy Bebop,0,0,1.000000
4,Mononoke Hime,0,0,0.859548
5,Howl no Ugoku Shiro,1,0,0.999675
6,Death Note,0,0,1.000000
7,Hotaru no Haka,0,1,0.999797
8,Toradora!,1,0,0.693459
9,Neon Genesis Evangelion,0,0,1.000000


## Interpretation

The model reaches around **60% accuracy** on 10 anime test set using only 50 labelled training samples, showing that semi-supervised label spreading can extract meaningful structure even from very sparse supervision.

Several well-known titles such as *Kimi no Na wa.*, *Fullmetal Alchemist: Brotherhood* and *Neon Genesis Evangelion* are correctly classified, indicating that the model captures consistent patterns for both optimistic and tragic narratives.

Most misclassifications are **emotionally ambiguous** cases such as *Howl's Moving Castle (Howl no Ugoku Shiro)* or *Spirited Away (Sen to Chihiro no Kamikushi)*. These titles mix hopeful visuals with darker themes, and the model appears to rely more on tonal and structural similarity than on final narrative closure. 

This reflects the inherent difficulty of defining “happy” versus “sad” in storytelling.
